In [ ]:
from numpy import get_include
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Khởi tạo trình duyệt
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Mở 1 trang sản phẩm




def get_csv(url):
    driver.get(url)
    time.sleep(0.5)
    # Lấy tên sản phẩm
    try:
        name = driver.find_element(By.TAG_NAME, "h1").text
        # Lấy giá
        def getprice():
            try:
                return driver.find_element(By.CLASS_NAME, "bs_price").find_element(By.TAG_NAME, "em").text
            except:
                return driver.find_element(By.CLASS_NAME, "box-price-present").text

        print("Tên:", name)
        print("Giá:", getprice())

        tag_exist = ['Cấu hình']

        def safe_click_get_info(driver, selector):
            try:
                element = driver.find_element(By.CSS_SELECTOR, selector)
                tab_name = element.find_element(By.TAG_NAME, "h3").text
                tag_exist.append(tab_name)
                driver.execute_script("arguments[0].click();", element)
            except:
                pass

        # Click các tab
        for i in range(2, 17):
            safe_click_get_info(driver, f'a[data-index="{i}"]')

        # Thu thập dữ liệu cấu hình
        specs_data = []
        uls = driver.find_elements(By.CSS_SELECTOR, "ul.text-specifi.active")
        for i, ul in enumerate(uls, start=1):
            lis = ul.find_elements(By.CSS_SELECTOR, "li")
            key_x = tag_exist[i-1]
            temp = []
            for li in lis:
                asides = li.find_elements(By.CSS_SELECTOR, "aside")
                if len(asides) >= 2:
                    key = asides[0].text.strip()[:-1]
                    # lấy toàn bộ text con trong aside[1] (phân cách bằng |)
                    values = [v.strip() for v in asides[1].text.split("\n") if v.strip()]
                    value = " | ".join(values)
                    temp.append((key, value))
            specs_data.append((key_x,temp))

        # Lấy ảnh
        def get_img(selector):
            results = []
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for el in elements:
                thumb = el.get_attribute("data-thumb")
                img = el.find_element(By.TAG_NAME, "img")
                src = img.get_attribute("src") or img.get_attribute("data-src")
                results.append((thumb, src))
            return results

        imgs = []
        for i in range(1, 7):
            imgs.extend(get_img(f"div.owl-item.slidenumber{i} .item-img"))

        def get_img_all():
            try:
                elements = driver.find_element(By.CSS_SELECTOR, "div.img-main.specification-img")
                return elements.find_element(By.CSS_SELECTOR,"img").get_attribute("src")
            except:
                pass
                return ''
            
        # Gộp dữ liệu
        data = {
            "Tên": name,
            "Giá": getprice(),
            "Cấu hình": "; ".join([f"{k}: {v}" for k, v in specs_data]),
            "Ảnh": "; ".join([src for thumb, src in imgs]),
            "Ảnh tổng thế": get_img_all()
        }
        return data
    except:
        pass
        return []
urls = []

def geturls(s):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(f"https://www.thegioididong.com/laptop-{s}")
    time.sleep(2)  # Chờ trang load, xử lý "Xem thêm"

    links = driver.find_elements(By.CSS_SELECTOR, "a[href^='/laptop/']")
    unique_links = []
    for a in links:
        href = a.get_attribute("href")
        if href and href.startswith("https://www.thegioididong.com/laptop/"):
            if href not in unique_links:
                unique_links.append(href)
        if len(unique_links) >= 30:
            break
    return unique_links

phone = ['apple-macbook','dell','asus','hp-compaq']
for i in phone:
    us = geturls(i)
    for u in us:
        urls.append(u)

# Xuất CSV
data = []
for url in urls:
    a = get_csv(url)
    if a != []:
        data.append(a)
df = pd.DataFrame(data)
df.to_csv("product_laptop.csv", index=False, encoding="utf-8-sig")


In [30]:
from numpy import get_include
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Khởi tạo trình duyệt
def geticon(s):
    driver.get(f"https://www.thegioididong.com/dtdd-{s}")
    time.sleep(2)  # Chờ trang load, xử lý "Xem thêm"

    names = driver.find_elements(By.CSS_SELECTOR, ".listproduct h3")

    name_product = []
    img_product = []
    for a in names:
        imgs = driver.find_element(By.CSS_SELECTOR, f'img[alt="{a.text}"]').get_attribute('src')
        if not imgs:
            imgs = driver.find_element(By.CSS_SELECTOR, f'img[alt="{a.text}"]').get_attribute('data-src')
        name_product.append(a.text)
        img_product.append(imgs)
    return name_product,img_product
phone = ['samsung','oppo','realme','vivo']
name_products = []
img_products = []
for i in phone:
    names,imgs = geticon(i)
    for name in names:
        name_products.append(name)
    for img in imgs:
        img_products.append(img)
driver.close()
df = pd.DataFrame(
    {
        'Name': name_products,
        'thumb': img_products
    }
)
df.to_csv('thumb_phone.csv',index = False)

In [29]:
df

,Name,thumb
0,Samsung Galaxy S25 FE 5G 8GB/256GB,https://cdn.tgdd.vn/Products/Images/42/342565/...
1,Samsung Galaxy A17 5G 8GB/128GB,https://cdn.tgdd.vn/Products/Images/42/341688/...
2,Samsung Galaxy A07 4GB/64GB,https://cdn.tgdd.vn/Products/Images/42/341802/...
3,Samsung Galaxy S25 Ultra 5G 12GB/256GB,https://cdn.tgdd.vn/Products/Images/42/333347/...
4,Samsung Galaxy Z Fold7 5G 12GB/256GB,https://cdn.tgdd.vn/Products/Images/42/338738/...
...,...,...
59,vivo V50 Lite 8GB/256GB,https://cdn.tgdd.vn/Products/Images/42/336702/...
60,vivo Y03 4GB/128GB,https://cdn.tgdd.vn/Products/Images/42/322996/...
61,vivo Y28 8GB/256GB,https://cdn.tgdd.vn/Products/Images/42/326023/...
62,vivo V40 5G 12GB/256GB,https://cdn.tgdd.vn/Products/Images/42/331985/...
